## ETL: Preparación de datasets

Extracción:  
En primera instancia obtendremos los datasets en crudo del siguiente repositorio: https://github.com/soyHenry/fe-ct-pimlops2  
O de la siguiente página: https://drive.google.com/drive/folders/1X_LdCoGTHJDbD28_dJTxaD4fVuQC9Wt5  
De estos links obtenremos dos archivos en formato .csv.  
Veremos paso a paso una guía breve para realizar una limpieza y obtener los datasets aptos para subir a un repositorio de github y que puedan ser consumidos por nuestra API a través de Render.
  
Una vez los archivos en nuestra computadora, procedemos a importar los archivos y a comenzar la transformación.  
  
Transformación:

In [24]:
import pandas as pd
import ast
import numpy as np
import json
import warnings
warnings.filterwarnings("ignore")

# Cargar el CSV en un DataFrame
df_movies = pd.read_csv('C:/Users/Usuario/Desktop/Data Science/SoyHenry/PI1_files/Datasets/movies_dataset.csv')
df_credits =pd.read_csv('C:/Users/Usuario/Desktop/Data Science/SoyHenry/PI1_files/Datasets/credits.csv')

In [25]:
#Visualizamos las dimensiones del dataframe movies, sus tipos, sus nulos y su peso.
print(df_movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [26]:
#Visualizamos las dimensiones del dataframe credits, sus tipos, sus nulos y su peso.
print(df_credits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
None


In [27]:
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Tenemos una lista con objetivos que cumplir en esta etapa de desarrollo del proyecto:

1) Desanidado de columnas:  
Identificar qué columnas están anidadas: belongs_to_collection, production_companies, genres, production_countries y spoken language. También será necesario desanidar el dataset Credits.

2) Tratamiento de valores nulos:  
Nulos cambiados por cero: Revenue y Budget.
Nulos a eliminar: Release.

3) Tratamiento de fechas:  
Formato pedido: AAAA-mm-dd  
  
4) Creación de columnas nuevas:  
Creación de columna release_year: año de la fecha de estreno.  
Creación de columna return: Retorno de inversión; revenue / budget (si no hay datos disponibles para calcularlo, deberá tomar el valor 0)

5) Eliminación de columnas:  
Video, imdb_id, adult, original_title, poster_path y homepage.

In [28]:
# 1) DESANIDADO DE COLUMNAS:

# Probamos técnicas para desanidado con una sola columna en un principio.
column_name = 'genres'

# Definimos una función que maneje NaN y cadenas mal formateadas
def safe_literal_eval(val):
    try:
        if pd.isna(val):
            return []
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return []

# Aplicamos la función a la columna para convertirla a listas de diccionarios
df_movies[column_name] = df_movies[column_name].apply(safe_literal_eval)

# Extraemos las IDs y Names en nuevas columnas
df_movies['genres_id'] = df_movies[column_name].apply(lambda x: [d['id'] for d in x])
df_movies['genres_name'] = df_movies[column_name].apply(lambda x: [d['name'] for d in x])

# Eliminamos la columna original si ya no es necesaria
df_movies.drop(columns=['genres'], inplace=True)

# Guardamos el resultado en un nuevo archivo CSV
#df_movies.to_csv('movies_dataset_transformado_2.csv', index=False)

print("Archivo CSV con columnas expandidas guardado como 'movies_dataset_transformado_2.csv'")

Archivo CSV con columnas expandidas guardado como 'movies_dataset_transformado_2.csv'


In [31]:
# Repetimos el proceso con las otras columnas
columns_to_process = {
    'belongs_to_collection': ['id', 'name', 'poster_path', 'backdrop_path'],
    'production_companies': ['id', 'name'],
    'production_countries': ['iso_3166_1', 'name'],
    'spoken_languages': ['iso_639_1', 'name']
}

# Procesamos cada columna en la lista
for column_name, keys in columns_to_process.items():
    # Convertimos la columna a listas de diccionarios
    df_movies[column_name] = df_movies[column_name].apply(safe_literal_eval)

    # Verificamos si hay valores inesperados en la columna
    invalid_values = df_movies[~df_movies[column_name].apply(lambda x: isinstance(x, list))][column_name]
    if not invalid_values.empty:
        print(f"Valores inesperados en la columna {column_name}:")
        print(invalid_values)
    
    # Extraemos las claves correspondientes y crear nuevas columnas
    for key in keys:
        df_movies[f'{column_name}_{key.upper()}'] = df_movies[column_name].apply(lambda x: [d.get(key, np.nan) for d in x if isinstance(d, dict)])

# Guardamos el resultado en un nuevo archivo CSV
# df_movies.to_csv('movies_dataset_transformado_final.csv', index=False)

print("Archivo CSV con columnas expandidas guardado como 'movies_dataset_transformado_final.csv'")

Archivo CSV con columnas expandidas guardado como 'movies_dataset_transformado_final.csv'


Hay columnas con valores booleanos! Esto no nos permitirá desarrollar el proceso de desanidado, siendo que son tres registros (contra 45K), procederemos a eliminarlos.

In [30]:
# Lista de índices de las filas que queremos eliminar
indices_to_remove = [19730, 29503, 35587]

# Eliminamos las filas correspondientes
df_movies = df_movies.drop(indices_to_remove)

# ^^^^^^^^^^Repetimos el proceso del cuadro anterior! (Después podemos seguir con la siguiente celda)^^^^^^^^^^

In [32]:
# Eliminamos la columna original si ya no es necesaria
df_movies.drop(columns=['belongs_to_collection','production_companies', 'production_countries', 'spoken_languages'], inplace=True)

# Guardamos el DataFrame limpio en un nuevo archivo CSV
#df_cleaned.to_csv('movies_dataset_registros_borrrados.csv', index=False)

print("Archivo CSV limpio guardado como 'movies_dataset_registros_borrrados.csv'")

Archivo CSV limpio guardado como 'movies_dataset_registros_borrrados.csv'


In [33]:
# SEGUIMOS CON EL DATASET CREDITS:
# Lo separaremos en dos datasets; Crew y Cast.

# CREW
# Filtramos filas donde 'crew' tenga longitud mayor a 0
df_valid_crew = df_credits[df_credits['crew'].str.len() > 0]

# Convertimos la columna 'crew' a listas de diccionarios si es necesario
df_valid_crew['crew'] = df_valid_crew['crew'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Expandimos diccionarios en filas separadas
df_crew_expanded = df_valid_crew.explode('crew').reset_index(drop=True)

# Convertimos los diccionarios en columnas
df_crew_expanded = pd.json_normalize(df_crew_expanded['crew'])

# Verificamos la longitud antes de repetir 'id'
expected_length = df_crew_expanded.shape[0]
repeated_ids = df_valid_crew.loc[df_valid_crew.index.repeat(df_valid_crew['crew'].str.len()), 'id'].values

# Si la longitud no coincide, ajustar
if len(repeated_ids) > expected_length:
    repeated_ids = repeated_ids[:expected_length]
elif len(repeated_ids) < expected_length:
    df_crew_expanded = df_crew_expanded.iloc[:len(repeated_ids)]

# Asignamos la columna 'id'
df_crew_expanded['id'] = repeated_ids

# Restablecemos el índice
df_crew_expanded.reset_index(drop=True, inplace=True)

print(df_crew_expanded.head())

                  credit_id department  gender   id         job  \
0  52fe4284c3a36847f8024f49  Directing     2.0  862    Director   
1  52fe4284c3a36847f8024f4f    Writing     2.0  862  Screenplay   
2  52fe4284c3a36847f8024f55    Writing     2.0  862  Screenplay   
3  52fe4284c3a36847f8024f5b    Writing     2.0  862  Screenplay   
4  52fe4284c3a36847f8024f61    Writing     0.0  862  Screenplay   

             name                      profile_path  
0   John Lasseter  /7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg  
1     Joss Whedon  /dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg  
2  Andrew Stanton  /pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg  
3      Joel Cohen  /dAubAiZcvKFbboWlj7oXOkZnTSu.jpg  
4    Alec Sokolow  /v79vlRYi94BZUQnkkyznbGUZLjT.jpg  


In [ ]:
# df_crew_expanded.to_csv('C:/Users/Usuario/Desktop/Data Science/SoyHenry/PI1/df_crew_expanded.csv', index=False)

In [34]:
# CAST
# Filtramos filas con longitud mayor a 0
df_valid_cast = df_credits[df_credits['cast'].str.len() > 0]

# Convertimos la columna 'cast' a listas de diccionarios si es necesario
df_valid_cast['cast'] = df_valid_cast['cast'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Expandimos diccionarios en filas separadas
df_cast_expanded = df_valid_cast.explode('cast').reset_index(drop=True)

# Convertimos los diccionarios en columnas
df_cast_expanded = pd.json_normalize(df_cast_expanded['cast'])

# Verificamos la longitud antes de repetir 'id'
expected_length = df_cast_expanded.shape[0]
repeated_ids = df_valid_cast.loc[df_valid_cast.index.repeat(df_valid_cast['cast'].str.len()), 'id'].values

# Si la longitud no coincide, ajustar
if len(repeated_ids) > expected_length:
    repeated_ids = repeated_ids[:expected_length]
elif len(repeated_ids) < expected_length:
    df_cast_expanded = df_cast_expanded.iloc[:len(repeated_ids)]

# Asignamos la columna 'id'
df_cast_expanded['id'] = repeated_ids

# Restablecemos el índice
df_cast_expanded.reset_index(drop=True, inplace=True)

print(df_cast_expanded.head())

   cast_id                character                 credit_id  gender   id  \
0     14.0            Woody (voice)  52fe4284c3a36847f8024f95     2.0  862   
1     15.0   Buzz Lightyear (voice)  52fe4284c3a36847f8024f99     2.0  862   
2     16.0  Mr. Potato Head (voice)  52fe4284c3a36847f8024f9d     2.0  862   
3     17.0       Slinky Dog (voice)  52fe4284c3a36847f8024fa1     2.0  862   
4     18.0              Rex (voice)  52fe4284c3a36847f8024fa5     2.0  862   

            name  order                      profile_path  
0      Tom Hanks    0.0  /pQFoyx7rp09CJTAb932F2g8Nlho.jpg  
1      Tim Allen    1.0  /uX2xVf6pMmPepxnvFWyBtjexzgY.jpg  
2    Don Rickles    2.0  /h5BcaDMPRVLHLDzbQavec4xfSdt.jpg  
3     Jim Varney    3.0  /eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg  
4  Wallace Shawn    4.0  /oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg  


In [ ]:
# df_cast_expanded.to_csv('C:/Users/Usuario/Desktop/Data Science/SoyHenry/PI1/df_cast_expanded.csv', index=False)

In [35]:
# 2) Tratamiento de valores nulos:

# Rellenamos los valores nulos de las columnas 'revenue' y 'budget' con 0
df_movies['revenue'].fillna(0, inplace=True)
df_movies['budget'].fillna(0, inplace=True)

# Eliminamos las filas donde 'release_date' es nulo
df_movies = df_movies.dropna(subset=['release_date'])

# 3) Tratamiento de fechas:

# Aseguramos que la columna release_date esté en formato datetime
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], errors='coerce')

# Convertimos la fecha al formato AAAA-mm-dd
df_movies['release_date'] = df_movies['release_date'].dt.strftime('%Y-%m-%d')

# 4) Creación de columnas nuevas:

# Creamos una nueva columna release_year extrayendo el año de release_date
df_movies['release_date'] = pd.to_datetime(df_movies['release_date']).dt.year

# Convertimos las columnas 'REVENUE' y 'BUDGET' a numéricas, forzando errores a NaN
df_movies['revenue'] = pd.to_numeric(df_movies['revenue'], errors='coerce')
df_movies['budget'] = pd.to_numeric(df_movies['budget'], errors='coerce')

# Creamos la columna 'RETURN' calculando REVENUE / BUDGET
df_movies['return'] = df_movies.apply(lambda x: x['revenue'] / x['budget'] if x['budget'] > 0 else 0, axis=1)

# 5) Eliminación de columnas:

# Eliminamos las columnas video, imdb_id, adult, original_title, poster_path y homepage.
df_movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)



In [21]:
df_movies

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,belongs_to_collection_NAME,belongs_to_collection_POSTER_PATH,belongs_to_collection_BACKDROP_PATH,production_companies_ID,production_companies_NAME,production_countries_ISO_3166_1,production_countries_NAME,spoken_languages_ISO_639_1,spoken_languages_NAME,return
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995,373554033.0,81.0,Released,NaN,...,[],[],[],[3],[Pixar Animation Studios],[US],[United States of America],[en],[English],12.451801
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,[],[],[],"[559, 2550, 10201]","[TriStar Pictures, Teitler Film, Interscope Co...",[US],[United States of America],"[en, fr]","[English, Français]",4.043035
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,[],[],[],"[6194, 19464]","[Warner Bros., Lancaster Gate]",[US],[United States of America],[en],[English],0.000000
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,[],[],[],[306],[Twentieth Century Fox Film Corporation],[US],[United States of America],[en],[English],5.090760
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,[],[],[],"[5842, 9195]","[Sandollar Productions, Touchstone Pictures]",[US],[United States of America],[en],[English],0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991,0.0,104.0,Released,NaN,...,[],[],[],"[7025, 10163, 16323, 38978]","[Westdeutscher Rundfunk (WDR), Working Title F...","[CA, DE, GB, US]","[Canada, Germany, United Kingdom, United State...",[en],[English],0.000000
45462,0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011,0.0,360.0,Released,NaN,...,[],[],[],[19653],[Sine Olivia],[PH],[Philippines],[tl],[],0.000000
45463,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003,0.0,90.0,Released,A deadly game of wits.,...,[],[],[],[6165],[American World Pictures],[US],[United States of America],[en],[English],0.000000
45464,0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917,0.0,87.0,Released,NaN,...,[],[],[],[88753],[Yermoliev],[RU],[Russia],[],[],0.000000


In [22]:
df_cast_expanded

,cast_id,character,credit_id,gender,id,name,order,profile_path
0,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,862,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,862,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,862,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,862,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,862,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
...,...,...,...,...,...,...,...,...
562469,12.0,"Dennis (segment ""The Wet Dream"")",55c8d0e7c3a36804a00009ab,2.0,227506,Collin Bernsen,2.0,None
562470,13.0,"Terri (segment ""Tango"")",55c8d103c3a3680408000ad0,0.0,227506,Allison Brooks,3.0,None
562471,14.0,"Lisa (segment ""Within Ten Minutes"")",55c8d126c3a3680408000ad9,0.0,227506,Elizabeth Burr,4.0,None
562472,15.0,"Lulu (segment ""Lulu"")",55c8d14492514173de000864,1.0,227506,Marianna Elliott,5.0,None


In [23]:
df_crew_expanded

,credit_id,department,gender,id,job,name,profile_path
0,52fe4284c3a36847f8024f49,Directing,2.0,862,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,52fe4284c3a36847f8024f4f,Writing,2.0,862,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,52fe4284c3a36847f8024f55,Writing,2.0,862,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,52fe4284c3a36847f8024f5b,Writing,2.0,862,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,52fe4284c3a36847f8024f61,Writing,0.0,862,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
...,...,...,...,...,...,...,...
464309,58e946ce9251414b28024571,Editing,0.0,67758,Editor,Ollie Rainbird,None
464310,52fe4c6c9251416c910f481b,Directing,2.0,67758,Director,Antonin Peretjatko,None
464311,52fe4c6c9251416c910f4821,Camera,0.0,227506,Director of Photography,Simon Roca,None
464312,52fe4c6c9251416c910f4827,Writing,2.0,227506,Writer,Antonin Peretjatko,None
